# Evolutionary Computing Practical Session 1 - SGA Solution

### Author: Buelent Uendes

The goal of this notebook is to make yourself a bit more familiar with how to solve specific 'simple' problems using evolutionary algorithms. Do not panic if you do not fully understand each of the algorithms, as some of them will be dealt with later during the course. This notebook should just give you more insight into how on can code up simple evolutionary algorithms to solve certain problems. Getting familiar with the core concepts will also help you with the assignments in the course.

This notebook will implement some parts of the lectures slides that are based on Chapter 1 - 4. 

As always, we need to first import several packages:

In [777]:
import numpy as np
from numpy.random import randint
from numpy.random import rand
import random
import matplotlib.pyplot as plt
from typing import List, Tuple

# np.random.seed(5)

**Instructions:**

In the notebook, you will see a couple of ToDos with some instructions. Try your best to work through them and to complete the notebook. In case you run into problems, do not hesitate to ask us or you can check out the solution notebook which is also uploaded on Canvas.

## Simple genetic algorithm (SGA)

The simple genetic algorithm (SGA) is one of the most important and well-known evolutionary algorithm and is often also used for benchmarking. For this reason, it is a good idea to get familiar with this popular method. 


To recall, here are the main features of the SGA algorithm:
    
- Representation: binary code.
- Crossover with probability $p_c$, and mutation with probability $p_m$.
- XCrossover, here we will use 1-point crossover. 
- Mutation is applied bitwise and independently.
- parent selection is proportional to fitness. Selection is implemented via Roulette Wheel algorithm.
- Replace the whole population with offspring (hence, survival selection is generational).


### 1.1 One-max problem 

In this problem, we will apply a SGA to the simple one-max problem. The one-max problem is a binary-string optimization problem in which one wants to maximize the number of ones contained in a string of bits. For instance, if the bitstring is of length 50, then the maximum of the one-max function is attained at a value of 50.

An important step is to initialize the population. The following function will generate a population of individual solutions.

In [778]:
#Define the population:

def init_population(n_population: int, bit_length:int) -> list:
    '''This returns a randomly initialized list of individual solutions of size n_population.'''
    
    '''
    ToDo:
        
    Please write a code that initializes a set of random solution to the problem.
    '''
        
    #WRITE YOUR CODE HERE!
    population = []
    for i in range(n_population):
        population.append(np.array(list(randint(2, size=bit_length))))
    return np.array(population)
        
        
     

In [779]:
population_test = init_population(10, 10)
population_test

array([[1, 0, 0, 1, 1, 0, 1, 0, 0, 1],
       [0, 0, 0, 0, 1, 1, 0, 0, 1, 1],
       [0, 1, 1, 1, 0, 0, 0, 0, 0, 1],
       [0, 1, 0, 1, 0, 1, 1, 0, 1, 0],
       [1, 0, 0, 0, 1, 0, 1, 0, 0, 1],
       [1, 0, 0, 0, 0, 1, 0, 0, 1, 1],
       [1, 1, 0, 0, 0, 1, 0, 1, 0, 0],
       [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
       [1, 1, 1, 0, 1, 1, 0, 0, 0, 1],
       [0, 1, 0, 1, 1, 0, 0, 0, 1, 1]])

In a next step, we will define the fitness function which is a crucial element of any evolutionary algorithm. It represents the environment and shapes the evolutionary process.

In [780]:
#Define the fitness function. The fitness function represents the environment.
def one_max(x:list) -> float:
    '''Takes a list of length bit_length and returns the sum of its elements.'''
    return np.sum(x)

Given this, we will create a function that returns the highest fitness value of the fittest individual(s).

In [781]:
def fittest_solution(fitness_function: callable, generation) ->float:
    '''This returns the highest fitness value of the whole generation.'''
    return np.max([fitness_function(generation[i]) for i in range(generation.shape[0])])

In the next step, we will define the mutation operator, i.e. here we will implement a bit-wise mutation.

In [782]:
#Define the mutation operator

# First define the bit flipping operator

def bit_flipping(x: list) -> list:
    
    '''
    ToDo:
        
    Please write a code that flips the bit, i.e. changes a 1 to a 0 and vice versa.
    '''
    
    if isinstance(x, (list or np.ndarray)):
        print(x)
        print(type(x))
        if x == 1:
            return 0
        else:
            return 1
        
    
    else:
        x_new = x.copy()
        x = np.array(x)
        if type(x) is list:
            x_new = x.copy()
            x = np.array(x)
            for index, element in enumerate(x):
                if element == 1:
                    x_new[index] = 0
                else:
                    x_new[index] = 1
        return np.array(x_new)



def mutation_operator(mutation_function: callable, p_mutation: float, x: list) ->np.ndarray:
    '''This function takes the mutation function and applies it 
    element-wise to the genes according to the mutation rate.'''
    
    return np.asarray([mutation_function(gene) if (np.random.uniform() <= p_mutation) else gene for gene in x])


In [783]:
original = [1, 0, 1, 1, 0, 0, 0, 1, 0, 0]
print(original)
mutated = bit_flipping(original)
print(mutated)

[1, 0, 1, 1, 0, 0, 0, 1, 0, 0]
[1, 0, 1, 1, 0, 0, 0, 1, 0, 0]
<class 'list'>
1


In [784]:
# print('original')
# print(np.array(population_test))
# print('mutated')
# print(mutation_operator(bit_flipping, 1, population_test))
# print('are they the same?')
# print(mutation_operator(bit_flipping, 1, population_test) == np.array(population_test))

In [785]:
def cross_over(parent_1: list, parent_2: list, p_crossover: float, p_uni:float = 0.5, uniform:bool=False) -> tuple:
    '''This function applies crossover for the case of two parents.'''
    
    #Check if cross-over is applied
    if p_crossover > np.random.uniform():
        #Random uniform crossover
        if uniform:  
            child_1 = []
            for gene in range(len(parent_1)):
                if p_uni > np.random.uniform():
                #Choose first parent
                    child_1.append(parent_1[gene])
                else:
                    child_1.append(parent_2[gene])
            
            # The second child is used by using an inverse mapping, 
            # We use the bit-flipping function defined above.
            child_2 = [bit_flipping(gene) for gene in child_1]
                
            return child_1, child_2
        
        #If no uniform crossover is selected, i.e. 1-point crossover is applied
        else:
            #We exclude the splitpoints in the beginning and the end
            split_point = randint(1, len(parent_1)-1)
            
            #Now return perform the one-point crossover
            child_1 = np.array([parent_1[gene] if gene <= split_point else parent_2[gene] 
                                for gene in range(len(parent_1))])
            child_2 = np.array([parent_2[gene] if gene <= split_point else parent_1[gene] 
                                for gene in range(len(parent_1))])
            
            return child_1, child_2            
    else:
        #Just returns the original parents
        return parent_1, parent_2
        

In [786]:
def selection_probabilities(generation, fitness_function: callable, sigma_scaling=False) -> list:
    '''
    Calculates the individual selection probabilities based on the fitness function. 
    Applies sigma-scaling if desired.
    '''
    
    number_individuals = generation.shape[0]
    total_fitness = np.sum([fitness_function(generation[i]) for i in range(number_individuals)]) 
    
    if sigma_scaling == True:
        
        mean_fitness = total_fitness/number_individuals
        std_fitness = np.std([fitness_function(generation[i]) for i in range(number_individuals)])
        c = 2 #Constant
        
        fitness_sigma = [np.max(fitness_function(generation[i])-(mean_fitness-(c*std_fitness)),0) for i 
                          in range(number_individuals)]
        
        #Now we need to sum up the sigma-scaled fitnesses
        total_fitness_sigma = np.sum(fitness_sigma)
        selection_prob = [fitness_sigma[i]/total_fitness_sigma for i in range(number_individuals)]
    else:
        #Apply normal inverse scaling
        selection_prob = [(fitness_function(generation[i])/total_fitness) for i in range(number_individuals)]
    return selection_prob

In [787]:
def cumulative_probability_distribution(selection_probability: list) -> list:
    '''Calculates the cumulative probability distribution based on individual selection probabilities.'''
    cum_prob_distribution = []
    current_cum_prob_dis = 0
    for i in range(len(selection_probability)):
        current_cum_prob_dis += selection_probability[i]
        cum_prob_distribution.append(current_cum_prob_dis)
    return cum_prob_distribution

In [788]:
def roulette_wheel_algorithm(cum_prob_distribution, number_of_parents = 2) -> list:
    '''
    Implements the roulette wheel algorithm as discussed in the 
    accompanying text book by Eiben and Smith (2015).
    '''

        
    '''
    ToDo:
        
    Please try to finish the implementation of the roulette_wheel_algorithm. 
    You will need the cum_prob_distribution function which is defined already above for you±
    '''    
    current_member = 1
    mating_pool = []
    while current_member <= number_of_parents:
        r = rand()
        i = 0
        while cum_prob_distribution[i] < r:
            i += 1
        mating_pool.append(i)
        current_member += 1
    
    return mating_pool    

In [789]:
test_cum_dist = [0, 0.1, 0.3, 0.6, 1.0]

print(roulette_wheel_algorithm(test_cum_dist, 2))

[4, 4]


## Run the simulation: SGA with Roulette Wheel Algorithm 

Now having implemented this, we can finally run our first genetic algorithm to optimize the `one_max` function!

In [790]:
#Define the hyperparameters, 
#following the recommendations presented in the textbook
#Eiben, A.E., Smith, J.E., Introduction to Evolutionary Computing., Springer, 2015, 2nd edition, page 100

#Define population size 
n_population = 20

#Define length of the bitstring
bit_length = 50

#Define mutation rate
p_mutation = 1/bit_length

#Crossover probability
p_crossover = 0.6

#Number of iterations
n_iter = 500

#Number of children
number_of_children = 2

#Initiliaze the generation
generation = init_population(n_population, bit_length)
best = fittest_solution(one_max, generation)
print('The current best solution in the initial generation is {0}'.format(best))

for i in range(1, n_iter+1):
    
    new_generation = []
    selection_prob_gen = selection_probabilities(generation, one_max)
    cum_prob_distribution = cumulative_probability_distribution(selection_prob_gen)
    #First step: Parent selection using roulette wheel algorithm

    #We loop over the number of parent pairs we need to get
    for j in range(int(n_population/number_of_children)):   
        
        mating_pool = roulette_wheel_algorithm(cum_prob_distribution)
            
        #Cross-over
        child_1, child_2 = cross_over(generation[mating_pool[0]], generation[mating_pool[1]], p_crossover, uniform=False)
            
        #Mutation for each child
        child_1 = mutation_operator(bit_flipping, p_mutation, child_1)
        child_2 = mutation_operator(bit_flipping, p_mutation, child_2)
            
        #Survival selection is here generational, hence all children replace their parents
        
        new_generation.append(child_1.tolist())
        new_generation.append(child_2.tolist())
            
    generation = np.asarray(new_generation)
    best = fittest_solution(one_max, generation)
    if i % 50 == 0:
        print('The current best population in generation {0} is {1}'.format(i, best))
        
    #Include a condition that stops the experiment when the optimal solution is found.
    if best == bit_length:
        print('---'*30)
        print('Done! The algorithm has found the optimal solution!')
        print('The current best population in generation {0} is {1}'.format(i, best))
        break
    

The current best solution in the initial generation is 31
The current best population in generation 50 is 35
The current best population in generation 100 is 35
The current best population in generation 150 is 35
The current best population in generation 200 is 35
The current best population in generation 250 is 35
The current best population in generation 300 is 35
The current best population in generation 350 is 35
The current best population in generation 400 is 35
The current best population in generation 450 is 35
The current best population in generation 500 is 35


In [791]:
print()

As we can see, the SGA does not fully solve the problem. One way to tackle this is to implement tournament selection, given its simplicity and its power to control the selection pressure (with varying k).

In [792]:
def tournament_selection(generation: list, fitness_function: callable, k: int) -> list:
    '''
    This implements the tournament selection. K random individual (with replacement) are 
    chosen and compete with each other. The index of the best individual is returned.
    '''
    
    # #First step: Choose a random individual and score it
    # number_individuals = generation.shape[0]
    # current_winner = randint(0, number_individuals)
    # #Get the score which is the one to beat!
    # score = fitness_function(generation[current_winner])

    # # random.choice(generation, size=k-1, replace=True)
    # for candidates in range(k-1): #We already have one candidate, so we are left with k-1 to choose

    #     current_candidate_index = randint(0, number_individuals)
    #     current_candidate = generation[current_candidate_index]
    #     candidate_score = fitness_function(current_candidate)

    #     #Compare the score of the candidate with the score of the current winner
    #     if candidate_score > score:
    #         current_winner = current_candidate_index

    #  First step: Choose a random individual and score it

    number_individuals = generation.shape[0]
    current_winner = randint(0, number_individuals)
    #Get the score which is the one to beat!
    score = fitness_function(generation[current_winner])

    # random.choice(generation, size=k-1, replace=True)
    for candidates in range(k-1): #We already have one candidate, so we are left with k-1 to choose

        # current_candidate_index = randint(0, number_individuals)
        current_candidate = generation[candidates]
        candidate_score = fitness_function(current_candidate)

        #Compare the score of the candidate with the score of the current winner
        if candidate_score > score:
            current_winner = candidates

    return current_winner
    

## Run the simulation: SGA with tournament selection

Now we can re-run the experiment from above, this time using tournament selection instead of fitness proportional selection implement via roulette wheel algorithm.

In [793]:
#Now we can re-run the experiment from above, this time using tournament selection:

#Define the hyperparameters, 
#following the recommendations presented in the textbook
#Eiben, A.E., Smith, J.E., Introduction to Evolutionary Computing., Springer, 2015, 2nd edition, page 100

#Define population size 
n_population = 20

#Define length of the bitstring
bit_length = 50

#Define mutation rate
p_mutation = 1/bit_length

#Crossover probability
p_crossover = 0.6

#Number of iterations
n_iter = 500

number_of_children = 2

#Tournament size
k = 5

#Initiliaze the generation
generation = init_population(n_population, bit_length)
best = fittest_solution(one_max, generation)
print('The current best solution in the initial generation is {0}'.format(best))

for i in range(1, n_iter+1):
    
    new_generation = []
    selection_prob_gen = selection_probabilities(generation, one_max)
    cum_prob_distribution = cumulative_probability_distribution(selection_prob_gen)
    #First step: Parent selection using roulette wheel algorithm

    #We loop over the number of parent pairs we need to get
    for j in range(int(n_population/number_of_children)):
        
        mating_pool = []
        for child in range(number_of_children):
            
            mate = tournament_selection(generation, one_max, k)
            mating_pool.append(mate)
            
        #Cross-over
        child_1, child_2 = cross_over(generation[mating_pool[0]], generation[mating_pool[1]], p_crossover, uniform=True)
            
        #Mutation for each child
        child_1 = mutation_operator(bit_flipping, p_mutation, child_1)
        child_2 = mutation_operator(bit_flipping, p_mutation, child_2)
            
        #Survival selection is here generational, hence all children replace their parents
        
        new_generation.append(child_1.tolist())
        new_generation.append(child_2.tolist())
            
    generation = np.asarray(new_generation)
    best = fittest_solution(one_max, generation)
    if i % 10 == 0:
        print('The current best population in generation {0} is {1}'.format(i, best))
        
    #Include a condition that stops when the optimal solution is found
    if best == bit_length:
        print('---'*20)
        print('Done! The algorithm has found the optimal solution!')
        print('The current best population in generation {0} is {1}'.format(i, best))
        break
        

The current best solution in the initial generation is 31
The current best population in generation 10 is 35
The current best population in generation 20 is 35
The current best population in generation 30 is 35
The current best population in generation 40 is 35
The current best population in generation 50 is 35
The current best population in generation 60 is 35
The current best population in generation 70 is 35
The current best population in generation 80 is 35
The current best population in generation 90 is 35
The current best population in generation 100 is 35
The current best population in generation 110 is 35
The current best population in generation 120 is 35
The current best population in generation 130 is 35
The current best population in generation 140 is 35
The current best population in generation 150 is 35
The current best population in generation 160 is 35
The current best population in generation 170 is 35
The current best population in generation 180 is 35
The current bes

As we can see, the tournament selection greatly improves the evolutionary algorithm for this particular problem!